In [1]:
from array import array
from skmultiflow.data import RegressionGenerator
from sail.models.torch.tcn import TCNRegressor
from sail.models.torch.tcn import TCN
import numpy as np
import torch

/Users/mehkj/miniconda3_i386/envs/imla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_samples = 6000
n_features = 12
stream = RegressionGenerator(random_state=1,
                             n_samples=n_samples,
                             n_features=n_features)
learner_tcn = TCNRegressor(c_in=n_features, c_out=1)


In [4]:
cnt = 0
y_true = array('d')
y_pred = array('d')
index = []
i = 0
wait_samples = 10
while cnt < n_samples and stream.has_more_samples():
    X, y = stream.next_sample(batch_size=wait_samples)
    y = np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], -1)
    y = y.reshape(y.shape[0], -1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # Test every n samples
    if (cnt % wait_samples == 0) & (cnt != 0):
        y_true.append(y[0])
        y_pred1 = learner_tcn.predict(X)[0][0]
        y_pred.append(y_pred1)
        index.append(i)
        i = i + 1
    learner_tcn.partial_fit(X, y)
    learner_tcn.predict(X)
    cnt += 1

     11           nan  0.0600
     12           nan  0.0423
     13           nan  0.0420
     14           nan  0.0417
     15           nan  0.0419
     16           nan  0.0416
     17           nan  0.0415
     18           nan  0.0416
     19           nan  0.0417
     20           nan  0.0416
     21           nan  0.0414
     22           nan  0.0416
     23           nan  0.0416
     24           nan  0.0415
     25           nan  0.0415
     26           nan  0.0415
     27           nan  0.0415
     28           nan  0.0417
     29           nan  0.0417
     30           nan  0.0415
     31           nan  0.0411
     32           nan  0.0414
     33           nan  0.0416
     34           nan  0.0418
     35           nan  0.0421
     36           nan  0.0422
     37           nan  0.0422
     38           nan  0.0419
     39           nan  0.0420
     40           nan  0.0418
     41           nan  0.0429
     42           nan  0.0420
     43           nan  0.0421
     44   

KeyboardInterrupt: 

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'iframe'

y_pred = y_pred.tolist()
y_true = y_true.tolist()
fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=index, y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()